In [2]:
import configparser
import pandas as pd
import re
# import and instantiate TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# import and instantiate the Logistic Regression model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

config = configparser.ConfigParser()
config.read('../config.cfg')
train_df = pd.read_csv(config['FILES']['TRAIN'])
test_df = pd.read_csv(config['FILES']['TEST'])
print(test_df.shape)
print(test_df['id'][0:5])

test_label = pd.read_csv(config['FILES']['LABEL'])
print(test_label.shape)
print(test_label['id'][0:5])

cols_name = ['obscene', 'insult', 'toxic', 'severe_toxic', 'identity_hate', 'threat']





(153164, 2)
0    00001cee341fdb12
1    0000247867823ef7
2    00013b17ad220c46
3    00017563c3f7919a
4    00017695ad8997eb
Name: id, dtype: object
(153164, 7)
0    00001cee341fdb12
1    0000247867823ef7
2    00013b17ad220c46
3    00017563c3f7919a
4    00017695ad8997eb
Name: id, dtype: object


In [3]:
def clean_text(text):
    text = text.lower()
    for change in [(r"what's", "what is "), (r"\'s", " "), (r"\'ve", " have "), (r"can't", "cannot "), 
                   (r"n't", " not "), (r"i'm", "i am "), (r"\'re", " are "), (r"\'d", " would "), (r"\'ll", " will "), (r"\'scuse", " excuse ")]:
        text = re.sub(change[0], change[1], text)
    text = text.strip(' ')
    return text


train_df['comment_text'] = train_df['comment_text'].map(lambda com: clean_text(com))
test_df['comment_text'] = test_df['comment_text'].map(lambda com: clean_text(com))

x_train = train_df.comment_text
x_test = test_df.comment_text
print(x_train.shape, x_test.shape)

# vect = TfidfVectorizer(max_df=60000, min_df=2, stop_words='english')
vect = TfidfVectorizer(max_features=5000, stop_words='english')
# learn the vocabulary in the training data, then use it to create a document-term matrix
x_train_tfid = vect.fit_transform(x_train)
# transform the test data using the earlier fitted vocabulary, into a document-term matrix
x_test_tfid = vect.transform(x_test)



(159571,) (153164,)


In [8]:
logreg = LogisticRegression(C=12.0)

# create submission file
submission_binary = pd.DataFrame(index=test_df['id'], columns=cols_name)
test_label_strip = test_label[test_label.toxic != -1]

for label in cols_name:
    print('... Processing {}'.format(label))
    y = train_df[label]
    yt = test_label_strip[label]
    # train the model using X_dtm & y
    logreg.fit(x_train_tfid, y)
    # compute the training accuracy
    y_pred_X = logreg.predict(x_train_tfid)
    y_pred_t = logreg.predict(x_test_tfid)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = logreg.predict_proba(x_test_tfid)[:, 1]
    submission_binary[label] = test_y_prob
    print('Test accuracy is {}'.format(accuracy_score(yt, y_pred_t[test_label_strip.index])))


... Processing obscene
Training accuracy is 0.9832300355327722
Test accuracy is 0.9632998843352403
... Processing insult
Training accuracy is 0.9755469352200588
Test accuracy is 0.9620338241270436
... Processing toxic
Training accuracy is 0.9639846839337975
Test accuracy is 0.9243021038481978
... Processing severe_toxic
Training accuracy is 0.9920787611784097
Test accuracy is 0.9924192691237613
... Processing identity_hate
Training accuracy is 0.9939713356436947
Test accuracy is 0.9903716902685298
... Processing threat
Training accuracy is 0.9981199591404453
Test accuracy is 0.9959204726624777
